## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from datetime import timedelta
from scipy.fftpack import fft
from sklearn.decomposition import PCA

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
import torch.optim as optim

## 모델 학습 사용 라이브러리
from tqdm.notebook import trange
import statistics
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'bearing'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['s1-c1',
 's1-c2',
 's1-c3',
 's1-c4',
 's1-c5',
 's1-c6',
 's1-c7',
 's1-c8',
 's2-c1',
 's2-c2',
 's2-c3',
 's2-c4',
 's3-c1',
 's3-c2',
 's3-c3',
 's3-c4']

## TAG Name format 변환 

* 위의 과정에서 Smart home dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

* s1-c5 Tag Name 사용하여 예제 진행

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = ['s1-c5']

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

's1-c5'
1


## Nasa Bearing Dataset 로드

* 데이터 로드시 train, validation, test 데이터 셋을 각각 Load

* 예제는 3번쨰 bearing의 이상탐지로 진행 -> 's1-c5' Tag Name 사용

* 나머지 상태 빼고 고장이 아니면 전부 정상으로 labeling 해서 진행

In [6]:
# 각 bearing의 상태를 시간 범위로 설정
B1 ={
    "early" : ["2003-10-22 12:06:24" , "2003-10-23 09:14:13"],
    "suspect" : ["2003-10-23 09:24:13" , "2003-11-08 12:11:44"],
    "normal" : ["2003-11-08 12:21:44" , "2003-11-19 21:06:07"],
    "suspect_1" : ["2003-11-19 21:16:07" , "2003-11-24 20:47:32"],
    "imminent_failure" : ["2003-11-24 20:57:32","2003-11-25 23:39:56"]
}
B2 = {
    "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
    "normal" : ["2003-11-01 21:51:44" , "2003-11-24 01:01:24"],
    "suspect" : ["2003-11-24 01:11:24" , "2003-11-25 10:47:32"],
    "imminient_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B3 = {
    "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
    "normal" : ["2003-11-01 21:51:44" , "2003-11-22 09:16:56"],
    "suspect" : ["2003-11-22 09:26:56" , "2003-11-25 10:47:32"],
    "Inner_race_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B4 = {
    "early" : ["2003-10-22 12:06:24" , "2003-10-29 21:39:46"],
    "normal" : ["2003-10-29 21:49:46" , "2003-11-15 05:08:46"],
    "suspect" : ["2003-11-15 05:18:46" , "2003-11-18 19:12:30"],
    "Rolling_element_failure" : ["2003-11-19 09:06:09" , "2003-11-22 17:36:56"],
    "Stage_two_failure" : ["2003-11-22 17:46:56" , "2003-11-25 23:39:56"]
}

## 데이터 전처리
   * hanning window, FFT, MinMaxScaling 적용

In [7]:
# 선택한 tag name 별 최대, 최소값 계산 함수
def set_minmax_value(table, name, start_time_train, end_time_train):
    
    # URL 인코딩
    start = quote(start_time_train)
    end = quote(end_time_train)
    
    # min max 데이터 로드
    df_ = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-scale.tql?table={table}&name={name}&start={start}&end={end}')
    
    ## Min , Max values 설정 
    Min = df_.iloc[:,1:-1].T
    Max = df_.iloc[:,2:].T
    
    return Min, Max 

In [8]:
# 데이터 시간 로드 함수
def time_data_load(table, name, start_time, end_time, timeformat):
    
    target = 'time'
    
    # 데이터 로드 
    df = pd.read_csv(f"http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?target={target}&table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}")
    
    # resample을 위해 임의의 value 컬럼 생성
    df['value'] = 0
    
    # resample 진행
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    df = df.resample('1s').mean()
    
    # 결측값 제거
    df = df.dropna()
    
    # 임의의 value 컬럼 제거
    df = df.drop(['value'], axis=1)
    
    return df

In [9]:
# 데이터 로드 함수
def data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # time 설정
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')

    # TIME을 기준으로 그룹화
    # 초 단위로 반올림
    filtered_data = df.copy()
    filtered_data.loc[:, 'TIME'] = filtered_data['TIME'].dt.floor('S')
    grouped = filtered_data.groupby('TIME')['s1-c5'].apply(list).reset_index()

    # 리스트를 개별 열로 나누기
    s1_c5_df = pd.DataFrame(grouped['s1-c5'].tolist())

    # 'TIME' 열과 병합
    result_df = pd.concat([grouped[['TIME']], s1_c5_df], axis=1)
    
    # label 설정
    # 각 채널데이터 비정상부분의 시간 값을 기준으로 label 설정 
    result_df['label'] = np.where((result_df['TIME'] >= "2003-11-25 10:57:32") & (result_df['TIME'] <= "2003-11-25 23:39:56"), 1, 0)
    
    result_df = result_df.drop(['TIME'], axis=1)
    
    result_df = result_df.dropna()
    
    return result_df

In [10]:
# 시간 변환 함수
# 시간 추가하려면 해당 과정 필요
# window_size : 데이터를 묶어서 수집되는 주기 
# step_size : 데이터의 간격 
def add_time(time_df, start_time, batch_size, window_size, step_size):
    
    # 몇개의 데이터를 로드해야 되는지 계산
    time = (batch_size * step_size)+ window_size - step_size - 1
    
    # 현재 시간의 인덱스 번호를 확인
    # 없는 경우는 맨처음 시간이 없는 경우이기 때문에 맨처음 인덱스로 지정함 
    try:
        index_now = time_df.index.get_loc(start_time)
    except KeyError:
        index_now = 0
    
    # 현재 시간 기준 배치 데이터의 마지막 시간 설정 
    end_time_ = str(time_df.index[index_now + time] + timedelta(seconds=1))
    
    # 다음 시작 시간의 인덱스 번호 설정
    index_next = index_now + time - abs(window_size - step_size - 1)
    
    # 다음 시작 시간 설정
    next_start_time_ = str(time_df.index[index_next])
    
    # URL 인코딩
    start_time_ = quote(start_time)
    end_time_ = quote(end_time_)
    next_start_time_ = quote(next_start_time_)
    
    return start_time_, end_time_, next_start_time_, index_next

In [11]:
# hanning window 함수 설정 
def set_hanning_window(sample_rate, df):
    
    # Hanning 윈도우 생성
    hanning_window = np.hanning(sample_rate)

    # 각 행에 Hanning 윈도우 적용
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

In [12]:
# FFT 변환 함수
def change_fft(sample_rate, df):
    # 신호의 총 샘플 수
    N = sample_rate
    
    # 각 행에 대해 FFT 적용
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    for i in range(df.shape[0]):
        # 각 행의 FFT 계산
        yf = fft(df.iloc[i].values)
        
        # FFT 결과의 절댓값을 계산하고 정규화 (유의미한 부분만)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # FFT 결과를 데이터 프레임으로 변환
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

In [13]:
def make_window(data, window_size, step_size):
    
    # 윈도우 슬라이딩 결과를 저장할 리스트
    windows = []
    labels = []
    
    # 슬라이딩 윈도우 적용
    for i in range(0, data.shape[0] - window_size + 1, step_size):
        window = data.iloc[i:i + window_size, :-1].values
        label_array = data.iloc[i:i + window_size, -1].values
        
        # 레이블 배열에서 하나라도 비정상 값이 있으면 레이블을 1로 설정
        if (label_array == 1).any():
            label = 1  
        else:
            label = 0
        
        windows.append(window)
        labels.append([label])
        
    return windows, labels    

## Min-Max Scaling 설정

* Process 컨셉상 전체 데이터를 불러오지 않기 때문에 최대 최소값을 사용하는 방식의 Min-Max Sclare를 설정 

In [14]:
# MinMaxScaler 클래스 정의
class MinMaxScaler:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    # 설정 파라미터 기반 scale 값 설정 
    def transform(self, X, min_values, max_values):
        X = np.array(X)
        self.min_ = np.array(min_values)
        self.max_ = np.array(max_values)
        
        if self.min_ is None or self.max_ is None:
            raise ValueError("Min and Max values are not set.")
        
        # scale 값이 0이되는 것을 막기 위해 1e-6 을 더해줌 
        scale = (self.max_ - self.min_) + 1e-6
        if np.any(scale == 0):
            raise ValueError("Min and Max values are the same, resulting in a scale of 0.")
        
        return (X - self.min_) / scale
    
    # 계산한 scale 값 기준 데이터 정규화 
    def fit_transform(self, X, min_values, max_values):
        """Set parameters and then transform X"""
        return self.transform(X,min_values, max_values)

## 학습 모델 설정 

* LSTM AE 기본 모델 사용

In [15]:
# LSTM Autoencoder 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # 인코더 LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, 2*hidden_dim)
        
        # 디코더 LSTM
        self.decoder_fc = nn.Linear(2*hidden_dim, hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)

    def forward(self, x):
        # 인코더 부분
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # 디코더 부분
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        return output

In [16]:
# 모델 설정 파라미터
# 입력 데이터 컬럼 수
# PCA 한값 
input_dim = 31

# LSMT hidden state 크기
hidden_dim = 15

# layer 수
num_layers = 3

# 학습률 
learning_rate = 0.01

# 모델 초기화
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(31, 15, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=15, out_features=30, bias=True)
  (decoder_fc): Linear(in_features=30, out_features=15, bias=True)
  (decoder_lstm): LSTM(15, 31, num_layers=3, batch_first=True)
)


## 모델 학습

* 필요한 배치 크기의 데이터만 불러와서 학습하는 방법으로 진행

In [17]:
# 모델 학습 함수 설정
def train(table, name, timeformat, model, start_time_train, end_time_train, batch_size, window_size, step_size, epochs, sample_rate, Min, Max, scaler, pca, time_df_train):
        
    # 초기 train loss 설정
    train_loss = []

    # 베스트 loss 초기화 
    best_Loss=np.inf

    for epoch in epochs:
        
        # 학습 모드 설정 
        model.train()
        
        # 초기 loss 초기화
        running_loss = 0.0
        total_step = 0
        
        # 초기 시작 시간 설정
        start_time_ = start_time_train
        
        # 끝 시간 설정
        end_time_train = str(time_df_train.index[-1])

        # while 문을 통해 데이터 호출 
        while start_time_ < end_time_train:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_, end_time_, next_start_time_, index_next= add_time(time_df_train, start_time_, batch_size, window_size, step_size)
            
            # 데이터 로드 
            data = data_load(table, name, start_time_, end_time_, timeformat)
            
            # hanning window 적용
            data_ = set_hanning_window(sample_rate, data.iloc[:,:-1])
            
            # FFT 적용 
            data_  = change_fft(sample_rate, data_ )
            
            # MinMax scaler 적용 
            data_ = scaler.fit_transform(data_, Min, Max)
            
            # PCA 적용
            data_ = pca.fit_transform(data_)
            
            # 데이터 프레임 + label 설정
            data_ = pd.DataFrame(data_)
            data_['label'] = data['label'].values

            # window 설정 
            windows, labels = make_window(data_ , window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data_) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows) == batch_size:
                
                # 총 배치수 체크용  
                total_step = total_step + 1
                
                # 데이터를 numpy 배열로 변환
                input_data = np.array(windows)

                # 데이터를 Tensor로 변환
                input_data = torch.tensor(input_data, dtype=torch.float32).to(device).float()

                # 옵티마이저 최적화 
                optimizer.zero_grad()
                
                # 모델 입력
                outputs = model(input_data)
                
                # loss 계산
                loss = criterion(outputs, input_data)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # 배치 리셋
                windows = []
            
            # 다음 시작 시간 설정    
            start_time_ = unquote(next_start_time_)
            
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next + (batch_size * step_size)+ window_size - step_size - 1 > len(time_df_train):
                break
        
        if total_step > 0:
            train_loss.append(running_loss / total_step)
            print(f'\ntrain loss: {np.mean(train_loss)}')
            
        if best_Loss > np.mean(train_loss):
            best_Loss = np.mean(train_loss)
            torch.save(model, f'./result/Nasa_Bearing_LSTM_AE_New_batch_비지도.pt')
            print('베스트 모델 저장') 
            
        epochs.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")
               
    return model

In [18]:
########################################### 학습 파라미터 설정 ################################################
# tag table 이름 설정
table = 'bearing'
# tag name 설정
name = quote(tags_, safe=":/")
# 학습 시작 시간 설정
start_time_train = '2003-10-22 12:06:24'
# 학습 끝 시간 설정
end_time_train = '2003-11-22 00:00:00'
# 시간 포멧 설정 
timeformat = quote('2006-01-02 15:04:05.000000')
# 배치 사이즈 설정
batch_size = 32
# window size 설정
window_size = 3
# step size 설정 -> window size 보다 크거나 같으면 안됨 
step_size = 1
# epoch 설정
epochs = trange(5, desc='training')
# 최대, 최소값 설정 
Min, Max = set_minmax_value(table, name, start_time_train, end_time_train)
# sample rate 설정
sample_rate = 20480
# Min-Max scaler 설정 
scaler = MinMaxScaler()
# PCA 설정
# 95%의 분산을 설명하는 주성분 선택
pca = PCA(n_components=31)
# 학습 시간 리스트 로드 
time_df_train = time_data_load(table, name, quote(start_time_train), quote(end_time_train), timeformat)

################################################ 학습 진행 ####################################################
train(table, name, timeformat, model, start_time_train, end_time_train, batch_size, window_size, step_size, epochs, sample_rate, Min, Max, scaler, pca, time_df_train)

training:   0%|          | 0/5 [00:00<?, ?it/s]


train loss: 0.0001948600776426314
베스트 모델 저장

train loss: 0.00010036500810252214
베스트 모델 저장

train loss: 6.882995255440574e-05
베스트 모델 저장

train loss: 5.305689636732798e-05
베스트 모델 저장

train loss: 4.359030993631071e-05
베스트 모델 저장


LSTMAutoencoder(
  (encoder_lstm): LSTM(31, 15, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=15, out_features=30, bias=True)
  (decoder_fc): Linear(in_features=30, out_features=15, bias=True)
  (decoder_lstm): LSTM(15, 31, num_layers=3, batch_first=True)
)

## 임계값 설정

* validation data를 사용하여 임계값 계산 
  * Max + K x 표준편차

In [19]:
# 베스트 모델 로드
model_ = torch.load(f'./result/Nasa_Bearing_LSTM_AE_New_batch_비지도.pt') 

In [20]:
def threshold_set(table, name, timeformat, model, start_time_valid, end_time_valid, batch_size, window_size, step_size, k, sample_rate, Min, Max, scaler,time_df_valid):
    valid_loss = []
    with torch.no_grad():
        
        model.eval()
         
        # 초기 시작 시간 설정
        start_time_v = start_time_valid
        
        # 끝 시간 설정
        end_time_valid = str(time_df_valid.index[-1])
        
        # while 문을 통해 데이터 호출 
        while start_time_v < end_time_valid:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_v, end_time_v, next_start_time_v, index_next_v = add_time(time_df_valid, start_time_v, batch_size, window_size, step_size)
            
            # 데이터 로드 
            data_v = data_load(table, name, start_time_v, end_time_v, timeformat)
            
            # hanning window 적용
            data_ = set_hanning_window(sample_rate, data_v.iloc[:,:-1])
            
            # FFT 적용 
            data_  = change_fft(sample_rate, data_ )
            
            # MinMax scaler 적용 
            data_ = scaler.fit_transform(data_, Min, Max)
            
            # PCA 적용
            data_ = pca.fit_transform(data_)
            
            # 데이터 프레임 + label 설정
            data_ = pd.DataFrame(data_)
            data_['label'] = data_v['label'].values

            # window 설정 
            windows_v, labels_v = make_window(data_ , window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data_) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows_v) == batch_size:
                
                # 데이터를 numpy 배열로 변환
                input_data_val = np.array(windows_v)
                
                # 데이터를 Tensor로 변환
                input_data_val = torch.tensor(input_data_val, dtype=torch.float32).to(device).float()

                # 모델 입력
                outputs_val = model(input_data_val)
                
                # loss 계산
                loss_val = criterion(outputs_val, input_data_val)
            
                valid_loss.append(loss_val.item())
                
                # 배치 리셋
                windows_v = []
                
            # 다음 시작 시간 설정    
            start_time_v = unquote(next_start_time_v)
            
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next_v + (batch_size * step_size)+ window_size - step_size - 1 > len(time_df_valid):
                break
            
            
        # 임계값 계산
        threshold =  max(valid_loss) + k * statistics.stdev(valid_loss)
      
                    
    return threshold

In [21]:
######################################## validation 파라미터 설정 #############################################

# 검증 시작 시간 설정
start_time_valid = '2003-11-22 00:00:00'
# 검증 끝 시간 설정
end_time_valid = '2003-11-23 13:00:00'
# 임계값 파라미터 설정
k = 10

# 검증 시간 리스트 로드
time_df_valid = time_data_load(table, name, quote(start_time_valid), quote(end_time_valid), timeformat)

################################################ 학습 진행 ####################################################
threshold = threshold_set(table, name, timeformat, model_, start_time_valid, end_time_valid , batch_size, window_size, step_size, k, sample_rate, Min, Max, scaler,time_df_valid)
print(threshold)

1.2756313173981703e-05


## 모델 테스트

In [22]:
## 테스트 함수 
## 테스트 함수 
def test(table, name, timeformat, model, start_time_test, end_time_test, batch_size, window_size, step_size, threshold, sample_rate, Min, Max, scaler, pca, time_df_test):
    test_loss = []
    labels = []
    with torch.no_grad():
        
        model.eval()
        
        # 초기 시작 시간 설정
        start_time_t = start_time_test
        
        # 끝 시간 설정
        end_time_test = str(time_df_test.index[-1])
        
        # while 문을 통해 데이터 호출 
        while start_time_t < end_time_test:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_t, end_time_t, next_start_time_t, index_next_t = add_time(time_df_test, start_time_t, batch_size, window_size, step_size)
            
            # 데이터 로드 
            data_t = data_load(table, name, start_time_t, end_time_t, timeformat)
            
            # hanning window 적용
            data_ = set_hanning_window(sample_rate, data_t.iloc[:,:-1])
            
            # FFT 적용 
            data_  = change_fft(sample_rate, data_ )
            
            # MinMax scaler 적용 
            data_ = scaler.fit_transform(data_, Min, Max)
            
            # PCA 적용
            data_ = pca.fit_transform(data_)
            
            # 데이터 프레임 + label 설정
            data_ = pd.DataFrame(data_)
            data_['label'] = data_t['label'].values

            # window 설정 
            windows_t, labels_t = make_window(data_ , window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data_) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows_t) == batch_size:
                
                # 데이터를 numpy 배열로 변환
                input_data_test = np.array(windows_t)
                input_data_label = np.array(labels_t)
                
                # 데이터를 Tensor로 변환
                input_data_test = torch.tensor(input_data_test, dtype=torch.float32).to(device).float()
                input_data_label = torch.tensor(input_data_label, dtype=torch.float32).to(device).long()
                
                # DataLoader 생성
                dataset = TensorDataset(input_data_test, input_data_label)
                data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
                
                for batch_input, batch_label in data_loader:

                    # 모델 입력
                    outputs_test = model(batch_input)
                    
                    # loss 계산
                    loss_test = criterion(outputs_test, batch_input)
                    
                    test_loss.append(loss_test.item())
                    labels.append(batch_label.item())
                
                # 배치 리셋
                windows_t = []
                
            # 다음 시작 시간 설정    
            start_time_t = unquote(next_start_time_t) 
            
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next_t + (batch_size * step_size)+ window_size - step_size - 1 > len(time_df_test):
                break
            
    # 최종 결과 생성 
    final_df = pd.DataFrame(test_loss, columns=['reconst_score'])
    final_df['label'] = labels

    # 각 임계값 별 label 설정 
    final_df['pred_label'] = np.where(final_df['reconst_score']>threshold,1,0)
    
    return final_df

In [23]:
######################################## 테스트 파라미터 설정 #############################################

# 검증 시작 시간 설정
start_time_test = '2003-11-23 13:00:00'
# 검증 끝 시간 설정
end_time_test = '2003-11-26 00:00:00'
# 검증 시간 리스트 로드
time_df_test = time_data_load(table, name, quote(start_time_test), quote(end_time_test), timeformat)

######################################## 테스트 진행 #############################################
final_df = test(table, name, timeformat, model_, start_time_test, end_time_test, batch_size, window_size, step_size, threshold, sample_rate, Min, Max, scaler, pca, time_df_test)

## 모델 성능 확인 

In [24]:
print(classification_report(final_df['label'], final_df['pred_label']))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       178
           1       0.93      1.00      0.97        14

    accuracy                           0.99       192
   macro avg       0.97      1.00      0.98       192
weighted avg       1.00      0.99      0.99       192



In [25]:
final_df['label'].value_counts()

label
0    178
1     14
Name: count, dtype: int64

In [26]:
final_df['pred_label'].value_counts()

pred_label
0    177
1     15
Name: count, dtype: int64

## 과적합 체크

* test 데이터뿐만 아니라 train, validation 데이터를 활용해 F1 score 계산
* 만약 train, validation 결과가 좋지 않다면 모델 과적합으로 판단

In [27]:
final_df_train = test(table, name, timeformat, model_, start_time_train, end_time_train, batch_size, window_size, step_size, threshold, sample_rate, Min, Max, scaler, pca, time_df_train)

In [28]:
print(classification_report(final_df_train['label'], final_df_train['pred_label'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1728

    accuracy                           1.00      1728
   macro avg       1.00      1.00      1.00      1728
weighted avg       1.00      1.00      1.00      1728



In [29]:
final_df_valid = test(table, name, timeformat, model_, start_time_valid, end_time_valid, batch_size, window_size, step_size, threshold, sample_rate, Min, Max, scaler, pca, time_df_valid)

In [30]:
print(classification_report(final_df_valid['label'], final_df_valid['pred_label'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       192

    accuracy                           1.00       192
   macro avg       1.00      1.00      1.00       192
weighted avg       1.00      1.00      1.00       192

